<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Postgress_Windows_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install

In [1]:
# Install postgresql server
#!sudo apt-get -y -qq update > /dev/null
!apt update > /dev/null
#!sudo apt-get -y -qq install postgresql > /dev/null
!apt install postgresql > /dev/null
#!sudo service postgresql start
!service postgresql start





 * Starting PostgreSQL 10 database server
   ...done.


## Configure

In [2]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#!sudo -u postgres psql -U postgres -c "ALTER USER tuser2 PASSWORD 'pw2';"
# Setup a database with name `sampledb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


#Windows Functions 
https://www.postgresqltutorial.com/postgresql-window-function/

In [3]:
!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c " \
CREATE TABLE product_groups (\
	group_id serial PRIMARY KEY,\
	group_name VARCHAR (255) NOT NULL\
);\
\
CREATE TABLE products (\
	product_id serial PRIMARY KEY,\
	product_name VARCHAR (255) NOT NULL,\
	price DECIMAL (11, 2),\
	group_id INT NOT NULL,\
	FOREIGN KEY (group_id) REFERENCES product_groups (group_id)\
);"

Password for user postgres: 
CREATE TABLE


In [4]:
!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c " \
INSERT INTO product_groups (group_name) \
VALUES \
	('Smartphone'), \
	('Laptop'), \
	('Tablet'); \
\
INSERT INTO products (product_name, group_id,price) \
VALUES \
	('Microsoft Lumia', 1, 200),\
	('HTC One', 1, 400),\
	('Nexus', 1, 500),\
	('iPhone', 1, 900),\
	('HP Elite', 2, 1200),\
	('Lenovo Thinkpad', 2, 700),\
	('Sony VAIO', 2, 700),\
	('Dell Vostro', 2, 800),\
	('iPad', 3, 700),\
	('Kindle Fire', 3, 150),\
	('Samsung Galaxy Tab', 3, 200);"

Password for user postgres: 
INSERT 0 11


In [5]:
!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c "select * from product_groups;"

Password for user postgres: 
 group_id | group_name 
----------+------------
        1 | Smartphone
        2 | Laptop
        3 | Tablet
(3 rows)



In [6]:
!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c "select * from products;"

Password for user postgres: 
 product_id |    product_name    |  price  | group_id 
------------+--------------------+---------+----------
          1 | Microsoft Lumia    |  200.00 |        1
          2 | HTC One            |  400.00 |        1
          3 | Nexus              |  500.00 |        1
          4 | iPhone             |  900.00 |        1
          5 | HP Elite           | 1200.00 |        2
          6 | Lenovo Thinkpad    |  700.00 |        2
          7 | Sony VAIO          |  700.00 |        2
          8 | Dell Vostro        |  800.00 |        2
          9 | iPad               |  700.00 |        3
         10 | Kindle Fire        |  150.00 |        3
         11 | Samsung Galaxy Tab |  200.00 |        3
(11 rows)



In [7]:
!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c "\
SELECT \
	AVG (price) \
FROM \
	products; \
"

Password for user postgres: 
         avg          
----------------------
 586.3636363636363636
(1 row)



In [8]:
!psql -h localhost -p 5432 -U postgres -W -d praxisdb -c "\
SELECT \
	group_name,\
	AVG (price)\
FROM\
	products\
INNER JOIN product_groups USING (group_id)\
GROUP BY\
	group_name;\
"

Password for user postgres: 
 group_name |         avg          
------------+----------------------
 Smartphone | 500.0000000000000000
 Tablet     | 350.0000000000000000
 Laptop     | 850.0000000000000000
(3 rows)



In [9]:
!psql -h localhost -p 5432 -U postgres -W -d praxisdb -c "\
SELECT\
	product_name,\
	price,\
	group_name,\
	AVG (price) OVER (\
	   PARTITION BY group_name\
	)\
FROM\
	products\
	INNER JOIN \
		product_groups USING (group_id);\
"

Password for user postgres: 
    product_name    |  price  | group_name |         avg          
--------------------+---------+------------+----------------------
 HP Elite           | 1200.00 | Laptop     | 850.0000000000000000
 Lenovo Thinkpad    |  700.00 | Laptop     | 850.0000000000000000
 Sony VAIO          |  700.00 | Laptop     | 850.0000000000000000
 Dell Vostro        |  800.00 | Laptop     | 850.0000000000000000
 Microsoft Lumia    |  200.00 | Smartphone | 500.0000000000000000
 HTC One            |  400.00 | Smartphone | 500.0000000000000000
 Nexus              |  500.00 | Smartphone | 500.0000000000000000
 iPhone             |  900.00 | Smartphone | 500.0000000000000000
 iPad               |  700.00 | Tablet     | 350.0000000000000000
 Kindle Fire        |  150.00 | Tablet     | 350.0000000000000000
 Samsung Galaxy Tab |  200.00 | Tablet     | 350.0000000000000000
(11 rows)



In [10]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-01-17 10:01:34.152206+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)